<a href="https://colab.research.google.com/github/RVS97/MNIST-GANs/blob/master/DISCeval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluate strength of different discriminators

#Import Libraries


In [0]:
import sys
import json
import os
import glob
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.optimizers import Adam, SGD
from tensorflow.python.keras.models import Sequential, Model, load_model
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose 
from tensorflow.python.keras.layers import Input, UpSampling2D, concatenate  
from tensorflow.python.keras.layers import LeakyReLU
import time
import tensorflow as tf
import numpy as np
import cv2
import random
from sklearn.utils import shuffle
from matplotlib import pyplot
from math import ceil
from scipy.stats import describe
from sklearn.model_selection import train_test_split
import time
import pickle
import datetime

# Google Drive Mount

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
rootDir = "/content/gdrive/My\ Drive/TCV2"
subrootDir = rootDir + "/Bonus"
!mkdir $subrootDir

def saveFile(filePath, rootDir):
  !cp $filePath $rootDir
  
def getFile(fileName, rootDir, localDir = "./"):
  path = rootDir + "/" + fileName
  !cp $path $localDir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
mkdir: cannot create directory ‘/content/gdrive/My Drive/TCV2/Bonus’: File exists


#Load Models

In [0]:
def getNoiseCGAN(size, seed=-1):
  numEls = size[0]
  labels = np.zeros(numEls, 'uint8')
  labelsPerClass = int(numEls/10)
  for i in range(1,10):
    labels[labelsPerClass*i:labelsPerClass*(i+1)] = i
  
  if seed != -1:
    np.random.seed(seed)
    
  return np.random.normal(size=size), to_categorical(labels, num_classes=10)

In [0]:
def getNoise(size):
    return np.random.normal(size=size), to_categorical(np.random.randint(0,10,(1,size[0])), num_classes=10)[0]

In [0]:
def getAccAndConf(predProbsAllOrg, gtLabelsAllOrg, batchSize=100):
  predProbsAll = np.copy(predProbsAllOrg)
  gtLabelsAll = np.copy(gtLabelsAllOrg)
  # predLables and gtLabels in one-hot encoding form
  N = len(predProbsAll)
  nClasses = len(predProbsAll[0])

  nBatches = int(N/batchSize)

  correctCount = 0
  confCorrect = 0
  confIncorrect = 0
  for batchId in range(nBatches):
    gtLabels = gtLabelsAll[batchId*batchSize:(batchId+1)*batchSize]
    predProbs = np.reshape(predProbsAll[batchId*batchSize:(batchId+1)*batchSize], (1, batchSize))

    # Get ground truth labels in decimal
    gtLabelsDec = gtLabels

    # Get classification labels
    predLabelsDec = np.copy(predProbs)

    predLabelsDec[predLabelsDec>=0.5]=1
    predLabelsDec[predLabelsDec<0.5]=0


    # Get indices of correctly/incorrect classified samples
    correctIdx0 = np.where(np.logical_and(predLabelsDec==0,gtLabelsDec==0))
    correctIdx1 = np.where(np.logical_and(predLabelsDec==1,gtLabelsDec==1))
    incorrectIdx0 = np.where(np.logical_and(predLabelsDec==1,gtLabelsDec==0))
    incorrectIdx1 = np.where(np.logical_and(predLabelsDec==0,gtLabelsDec==1))

    # Get accuracy
    nCorrect = len(correctIdx0[0]) + len(correctIdx1[0])
    nIncorrect = len(incorrectIdx0[0]) + len(incorrectIdx1[0])
    correctCount += nCorrect
    #print(correctCount)

    # Get samples that have been correctly/incorrectly classified
    correctSamples0 = predProbs[correctIdx0]
    correctSamples1 = predProbs[correctIdx1]
    incorrectSamples0 = predProbs[incorrectIdx0]
    incorrectSamples1 = predProbs[incorrectIdx1]

    # Get confidences
    if nCorrect != 0:
      confCorrect += np.sum(correctSamples1) + np.sum(1-correctSamples0)
      #print(confCorrect)

    if nIncorrect != 0:
      confIncorrect += np.sum(incorrectSamples1) + np.sum(1-incorrectSamples0)

  acc = correctCount/N
  if correctCount == 0:
    confCorrect = 0
  else:
    confCorrect = confCorrect/correctCount
  
  if N-correctCount == 0:
    confIncorrect = 0
  else:
    confIncorrect = confIncorrect/(N-correctCount)
    
  return acc, confCorrect, confIncorrect

In [0]:
# folders = ['07_11h_35m','07_12h_09m','07_12h_54m','07_13h_53m','07_14h_53m','07_16h_09m'] # Deep network
folders = ['07_11h_56m','07_12h_32m','07_13h_10m','07_17h_06m','07_17h_44m','07_18h_29m'] # Shallow network

In [0]:
import os

# strong discriminator
targetDir = "/content/gdrive/My Drive/TCV2/cGAN2" #1=deep
folder = folders[3]


# Set number of data samples to test
numEls = 10000

# Set target epoch number
targetEpoch = 15

# Get noise input
# noiseVec, fakeLabels = getNoiseCGAN((numEls, 100), seed=1010101)
noiseVec, fakeLabels = getNoise((numEls, 100))

# Download strong model
targetFolder = "\"" + targetDir + "/" + folder + "\""
targetFile = folder + "_gan_" + str(targetEpoch) + ".h5"
getFile(targetFile, targetFolder)

# Load gan
model = load_model(targetFile)

# Load generator
gen = model.layers[2]


In [0]:
print(values.shape)
valuesAccum = np.zeros((10000,1))
print(valuesAccum.shape)
valuesAccum[2] = 3
print(valuesAccum[2])

(10000, 1)
(10000, 1)
[3.]


In [0]:

losses = np.zeros(len(folders))
valuesAccum = np.zeros((10000,1))

for i,f in enumerate(folders):
  print(f)
  # load other GAN models
  targetFolder = "\"" + targetDir + "/" + f + "\""
  targetFile = f + "_gan_" + str(targetEpoch) + ".h5"
  getFile(targetFile, targetFolder)

  model = load_model(targetFile)
  # Extract discriminator and merge with generator
  disc = model.layers[3]
  
  noiseInput = Input(shape=(100,))
  labelInput = Input(shape=(10,))
  
  genGAN = gen([noiseInput, labelInput])
  out = disc([genGAN, labelInput])
  gan = Model([noiseInput, labelInput], out, name="gan")
  
  # Evaluate GAN loss on noise
  values = gan.predict([noiseVec, fakeLabels])
  valuesAccum += values
  [acc, confC, confI] = getAccAndConf(values, np.zeros(10000))
  
  losses[i] = acc
  print(acc)
valuesAccum = valuesAccum/len(folders)
[acc2, confC2, confI2] = getAccAndConf(valuesAccum, np.zeros(10000))
losses

07_11h_56m
0.9493
07_12h_32m
0.6912
07_13h_10m
0.5178
07_17h_06m
0.4353
07_17h_44m
1.0
07_18h_29m
0.9994


array([0.9493, 0.6912, 0.5178, 0.4353, 1.    , 0.9994])

In [0]:
acc2

0.9902